## Systemd

- Purpose of an init system is:
    - Initialize the components that must be started after the Linux kernel is booted.
    - Manage services and daemons at any point while the system is running.


- systemd features
    - Backwards compatible with SysV
    - Parallel startup of system services at boot time,
    - On-demand activation of daemons
    - Dependency-based service control logic


- Units
    - Units are resources that systemd knows how to manage. 
    - They are categorized by the type of resource they represent.


        - ex daemons are known as service units. *.service

### Available Units

In [12]:
# Available units
systemctl -t help # --type help

Available unit types:
service
socket
busname
target
device
mount
automount
swap
timer
path
slice
scope


#### Different type of units:

### Targets

Used to group other units together and to bring the system to certain states. (Runlevel)

Default target:

In [13]:
systemctl get-default

graphical.target


In [ ]:
set-default graphical.target

Default target paths:

In [5]:
ll /etc/systemd/system/default.target
ll /lib/systemd/system/default.target

lrwxrwxrwx 1 root root 36 Dec  9 03:30 /etc/systemd/system/default.target -> /lib/systemd/system/graphical.target
lrwxrwxrwx 1 root root 16 Nov 25 16:27 /lib/systemd/system/default.target -> graphical.target


Going into a target:

In [ ]:
# AllowIsolate=yes (isolation must be yes)
systemctl isolate graphical.target # same as: telinit runlevel

+ rescue = 1 user
> "rescue" is equivalent to the old runlevel 1
> everything is mounted, but no normal services are started

+ emergency = nothing but systemd
> No services are started, no mount points mounted
> no sockets established, good for debugging step by step

In [ ]:
systemctl rescue # like isolate rescue but sends a wall message
systemctl --no-wall rescue

#### Go into recovery (rescue) mode by editing grub:

In [ ]:
systemd.unit=rescue.target # same as old school: 1 at the end of kernel line

#### Managing services

In [ ]:
systemctl start name.service
systemctl stop name.service

# Stop, then start again.
systemctl restart name.service

# Restarts a service only if it is running.
systemctl try-restart name.service

##### Reload
If the application in question is able to reload its configuration files (without restarting), you can issue the reload command to initiate that process.

In [ ]:
# Reloads configuration.
systemctl reload name.service

# try to reload if can't restarts
systemctl reload-or-restart name.service
systemctl try-reload-or-restart name.service

#### Check service status

In [ ]:
systemctl status name.service

systemctl is-active name.service # running
systemctl is-enabled name.service # will be activated when booting
systemctl is-failed name.service # tried to active but failed

#### Change service status

In [ ]:
systemctl enable name.service
systemctl disable name.service

# combination of disable and enable and is useful to reset the symlinks
systemctl reenable name.service 

#### Mask and unmasking
you can mask any service unit to prevent it from being started manually or by another service:

In [ ]:
# links /etc/systemd/system/name.service -> /dev/null
systemctl mask name.service
systemctl unmask name.service

#### List units 

In [ ]:
# list (all kind of units) [loaded and active]
systemctl list-units

# --all = active or not (inactive, dead, not-found, etc)

# list (all kind of units) [loaded] active or not
systemctl list-units --all

# list (service units) [loaded and currently active] 
systemctl list-units --type=service

# list (target units) [loaded] active or not 
systemctl list-units --type=target --all

# list (service units) [loaded but currently inactive]
# if we don't use --all it's only shows active ones
systemctl list-units --all --type=service --state=inactive

# list (target units) [ltried to load but not found] 
systemctl list-units --all --type=target --state=not-found

list-units only displays units that systemd has **attempted to load** (wanted by).  
To see every available unit file including those that systemd has not attempted to load:

In [ ]:
# list all [available] service units to see if they are enabled or not
systemctl list-unit-files --type service # can be used for other units like: targets etc.

In [ ]:
systemd-networkd-resolvconf-update.path    static 
accounts-daemon.service                    enabled
alsa-utils.service                         masked # /dev/null
console-getty.service                      disabled

In [17]:
systemctl list-unit-files --state masked | head -3

UNIT FILE                    STATE 
alsa-utils.service           masked
bootlogd.service             masked


***static***: unit file does not contain an "install" section. these units cannot be enabled and these are used only as a dependency of another units.

In [3]:
vimcat /lib/systemd/system/systemd-networkd-resolvconf-update.path
# vimcat /lib/systemd/system/colord.service

[Unit]
Description=Trigger resolvconf update for networkd DNS
ConditionPathIsSymbolicLink=/etc/resolv.conf

[Path]
PathChanged=/run/systemd/netif/state


In [5]:
vimcat /lib/systemd/system/accounts-daemon.service # systemctl cat accounts-daemon.service

[Unit]
Description=Accounts Service

# In order to avoid races with identity-providing services like SSSD or
# winbind, we need to ensure that Accounts Service starts after
# nss-user-lookup.target
After=nss-user-lookup.target
Wants=nss-user-lookup.target

[Service]
Type=dbus
BusName=org.freedesktop.Accounts
ExecStart=/usr/lib/accountsservice/accounts-daemon

[Install]
# We pull this in by graphical.target instead of waiting for the bus
# activation, to speed things up a little: gdm uses this anyway so it is nice
# if it is already around when gdm wants to use it and doesn't have to wait for
# it.
WantedBy=graphical.target


#### List dependecies

In [ ]:
systemctl list-dependencies multiuser.target
systemctl list-dependencies --after *.service/target
systemctl list-dependencies --before *.service/target

#### Other functionalities

In [ ]:
# Alias
systemctl show multi-user.target -p Names
 ⁠
# Controlling systemd on a Remote Machine
systemctl --host user_name@host_name command

- - -
## Lecture notes

- All lecture notes are available at github in `ipynb` format: [https://github.com/ravexina/linux-notes](https://github.com/ravexina/linux-notes). 


- There are `html` exports of project available at: [https://ravexina.github.io/linux-notes](https://ravexina.github.io/linux-notes).

## License
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
</a>

<span property="dct:title">Linux Notes</span> by 
<a href="http://github.com/ravexina/Linux-Notes" property="cc:attributionName" rel="cc:attributionURL">Milad As (Ravexina)</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

- - -

<a rel="license" href="https://gitlab.com/ravexina/"><img alt="ravexina's gitlab" style="display:inline;border-width:0;height:30px;float:left;" src="img/logo/logo-square.png" /></a> 

<a rel="license" href="https://github.com/ravexina/"><img alt="ravexina's github" style="float:right;display:inline;border-width:0;height:30px;" src="img/logo/GitHub_Logo.png" /></a>